# Anatomy of a neural network: Understanding core Keras APIs

## Processamento de Dados em Tensores

Diferentes tipos de dados são armazenados em tensores de diferentes dimensões e processados por camadas apropriadas em redes neurais:

- **Dados vetoriais simples**: Armazenados em tensores de rank-2 com forma `(samples, features)`. Processados por camadas densamente conectadas (*Dense*).
- **Dados sequenciais**: Armazenados em tensores de rank-3 com forma `(samples, timesteps, features)`. Processados por camadas recorrentes (*LSTM*) ou convolucionais 1D (*Conv1D*).
- **Dados de imagem**: Armazenados em tensores de rank-4. Processados por camadas convolucionais 2D (*Conv2D*).

## Estrutura da API do Keras: A Classe Layer

Uma API simples deve ter uma única abstração central. No Keras, essa abstração é a classe **Layer**. Tudo no Keras é uma **Layer** ou interage diretamente com uma.

- **O que é uma Layer?**  
  - Um objeto que encapsula **estado** (pesos) e **cálculo** (forward pass).
  - Os pesos são geralmente definidos no método `build()` (ou podem ser criados no construtor `__init__()`).
  - O cálculo é implementado no método `call()`.


In [11]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

#### Construção de camada densa (fully connected layer) personalizada no Keras. Essa camada funciona como um neurônio artificial: ela recebe uma entrada, multiplica por pesos, soma um viés (bias) e aplica uma ativação.

In [ ]:
class SimpleDense(keras.layers.Layer):
    def __init__(self, units, activation=None):
        super().__init__() # chama o construtor da classe Layer
        self.units = units # units = qtd de neuronios
        self.activation = activation # transforma a saída para ajudar na aprendizagem

    def build(self, input_shape): # build serve para criar pesos da camada
        input_dim = input_shape[-1]
        self.W = self.add_weight(shape=(input_dim, self.units), initializer="random_normal")
        self.b = self.add_weight(shape=(self.units,), initializer="zeros")

    def call(self, inputs): # call faz o calculo da saida da camada
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y

In [5]:
my_dense = SimpleDense(units=32, activation=tf.nn.relu)
input_tensor = tf.ones(shape=(2, 784))
output_tensor = my_dense(input_tensor)
print(output_tensor.shape)

(2, 32)


#### Layers definidas no Keras 

Possuem inferência automática de formato

In [7]:
from tensorflow.keras import models, layers

model = models.Sequential([
    layers.Dense(32, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(10, activation="softmax")
])

-----

## **From Layers to Models**

### **1. Modelos de Deep Learning**
Um modelo de deep learning é essencialmente um **grafo de camadas**. No Keras, esse conceito é representado pela classe `Model`. 

### **2. Tipos de Arquiteturas de Redes Neurais**
Há uma variedade maior de topologias de redes. Algumas das mais comuns são:
- **Redes de duas ramificações (Two-branch networks)**
- **Redes multihead (Multihead networks)**
- **Conexões residuais (Residual connections)**

O design da topologia de uma rede pode se tornar bastante complexo. Um exemplo clássico é a arquitetura **Transformer**, usada para processar dados textuais.

### **3. Como Construir Modelos no Keras**
Existem **duas abordagens principais** para construir modelos no Keras:
1. **Subclasse da classe `Model`** → Criamos modelos personalizados estendendo a classe `Model`.
2. **Functional API** → Permite construir modelos mais flexíveis e reutilizáveis com menos código.

Ambas as abordagens serão exploradas mais a fundo nos notebooks seguintes.

### **4. A Topologia do Modelo e o Espaço de Hipóteses**
A topologia do modelo é como o design da rede neural—ou seja, como as camadas são organizadas e conectadas. Isso define um espaço de hipóteses, que é o conjunto de soluções possíveis que o modelo pode aprender a partir dos dados.

O modelo aprende recebendo um sinal de feedback (o erro entre a previsão e o valor real) e ajusta seus pesos para melhorar as previsões.

-----

### 🛠️ `compile()`: Configurando o Modelo  
Antes de treinar uma rede neural no Keras, precisamos configurar como ela será otimizada. O método `compile()` define:  

1. **Otimizador**: Controla como os pesos da rede serão ajustados (ex: `adam`, `sgd`).  
2. **Função de perda**: Mede o erro entre as previsões e os valores reais (ex: `categorical_crossentropy`, `mse`).  
3. **Métricas**: Acompanha o desempenho durante o treinamento (ex: `accuracy`).  


In [ ]:
model = keras.Sequential([keras.layers.Dense(1)])

# importante escolher um otimizador, uma loss e as métricas
model.compile(optimizer="rmsprop",
 loss="mean_squared_error",
 metrics=["accuracy"])

### 📊 `fit()`: Treinando o Modelo  
Após compilar, usamos `fit()` para **treinar** a rede neural com os dados. Esse método ajusta os pesos da rede para minimizar o erro da função de perda.  

🔹 **Parâmetros principais:**  
- `x` e `y`: Dados de entrada e saída.  
- `epochs`: Número de vezes que a rede verá os dados completos.  
- `batch_size`: Quantos exemplos são processados antes de atualizar os pesos.  
- `validation_data`: Dados para avaliar o desempenho durante o treinamento.  

In [12]:
inputs = np.random.rand(1000, 20)  
targets = np.random.randint(0, 2, size=(1000,))  # 1000 saídas binárias (0 ou 1)

model = keras.Sequential([
    layers.Dense(32, activation="relu", input_shape=(20,)),
    layers.Dense(1, activation="sigmoid")  # saída binária
])

# compilando
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# treinando o modelo com fit
history = model.fit(inputs, targets, epochs=5, batch_size=128)

print(history.history)

c:\Users\BeatrizAlmeida\anaconda3\envs\redes-neurais\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.4662 - loss: 0.7313
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5054 - loss: 0.7004 
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4881 - loss: 0.7040
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5182 - loss: 0.6980 
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4906 - loss: 0.6988 
{'accuracy': [0.4830000102519989, 0.5009999871253967, 0.5040000081062317, 0.5009999871253967, 0.5059999823570251], 'loss': [0.7173683047294617, 0.7016859650611877, 0.6998783946037292, 0.699131965637207, 0.6971515417098999]}
